# 1. Retriever

In [1]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import BedrockEmbeddings 
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1")

model_kwargs = { 
    "max_tokens_to_sample": 1024, 
    "temperature": 0, 
    "top_p": 0.5, 
    "stop_sequences": ["Human:"]
}

default_model_id = "anthropic.claude-v2"
bedrock_base_kwargs = dict(model_id=default_model_id, model_kwargs= model_kwargs)


In [2]:
persist_directory = 'streamlit/docs/chroma/'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=bedrock_embeddings)

In [6]:
print(vectordb._collection.count())
question = "Como se compone el Congreso Nacional?"

results = vectordb.max_marginal_relevance_search(query=question, k=8)

884


In [16]:
len(results[7].page_content)

1958

# 2. Chatbot

In [51]:
from langchain.chat_models import BedrockChat
llm = BedrockChat(**bedrock_base_kwargs)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    return_source_documents=True,
    verbose=True, 
    retriever=vectordb.as_retriever(search_type = "mmr", search_kwargs={"k": 10})
)

In [52]:
from langchain.prompts import PromptTemplate

promp_template = """
Responde a la siguiente pregunta tan preciso como sea posible empleando el contexto encerrado por ##, 
si la respuesta no esta contenida en el contexto o si el contexto esta vacio responde "No lo se".
##{context}##

Question:{question}
"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=promp_template)

qa_chain.combine_documents_chain.llm_chain.prompt =QA_CHAIN_PROMPT



In [56]:
result = qa_chain({"query": "existe paridad en la propuesta de constitución?"})
print(result['query'],result['result'])
result = qa_chain({"query": "existe igualdad hombre mujer en la propuesta de constitución?"})
print(result['query'],result['result'])
result = qa_chain({"query": "Puedo ser presidente si tengo 16 años?"})
print(result['query'], result['result'])
result = qa_chain({"query": "Cuáles otros requisitos hay para ser presidente?"})
print(result['query'], result['result'])
result = qa_chain({"query": "Como se compone el Congreso Nacional?"})
print(result['query'],result['result'])
result = qa_chain({"query": "Qué requisitos existen para ser Senador?"})
print(result['query'],result['result'])





> Entering new RetrievalQA chain...

> Finished chain.
existe paridad en la propuesta de constitución?  Según el contexto provisto, no se menciona explícitamente si existe paridad en la propuesta de constitución. El contexto se refiere a los procedimientos de cambio constitucional y al procedimiento de reemplazo constitucional, pero no especifica si la propuesta constitucional contempla paridad. Dado que el contexto está incompleto, no puedo determinar de forma definitiva si la propuesta constitucional chilena incluye paridad o no.


> Entering new RetrievalQA chain...

> Finished chain.
existe igualdad hombre mujer en la propuesta de constitución?  Según el contexto provisto, la propuesta de constitución sí promueve la igualdad entre hombres y mujeres en los siguientes artículos:

- Artículo 3.2: "La ley promoverá el acceso igualitario de mujeres y hombres a los mandatos electorales y cargos de elección popular, así como su participación en condiciones de igualdad en los distintos á

In [55]:
result['query']

{'query': 'existe paridad hombre mujer en la propuesta de constitución?',
 'result': ' Según el contexto provisto, no se menciona explícitamente la paridad hombre-mujer en la propuesta de constitución. El texto se refiere a la participación política y representación en general, pero no especifica la paridad de género. Por lo tanto, la respuesta es: no lo sé.',
 'source_documents': [Document(page_content='96 \n  \nmediante decre to supremo exento, a un plebiscito nacional constitucional para que el \nelectorado se pronuncie sobre la propuesta.  \n \n6. Las reformas constitucionales que modifiquen este artículo deberán ser aprobadas por los \ndos tercios de los diputados y senadores en ejer cicio.  \n \nArtículo 2 19 \n \nA los referendos a que hace referencia esta Constitución les serán aplicables las normas sobre \nplebiscitos que establezcan las leyes.', metadata={'page': 95, 'source': 'docs/Texto-aprobado-Consejo-Constitucional_06.10.23.pdf'}),
  Document(page_content='favorables de 

# 3. with Memory

In [37]:
from langchain.prompts import PromptTemplate
template = """
You are a constitutional law assitant who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december 17th 2023.
(when asked for the source, provide the following link https://www.procesoconstitucional.cl/wp-content/uploads/2023/10/Texto-aprobado-Consejo-Constitucional_06.10.23.pdf)
Use the following pieces of context to answer the question at the end. Provide page number, article and letter if it is part of the answer. 
Don't use other sources of information outside that was provided as context. Don't user your prior knowledge about laws. Don't make up the answer using your intrinsic knowledge.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer. 
If the user doesn't ask a specific question (like greetings, goodbyes or thanking you) just reply the casual conversation. 
Never take a stand about approving or rejecting the draft, invite the user to read the draft and make his/her own informed opinion.
Always validate at the end if the answer was helpful to the user.

{context}
Question: {question}
Assistant:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)


In [23]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    ai_prefix="A",
    human_prefix="B"
)


In [38]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_type = "mmr",  search_kwargs={"k": 8} ),
    return_source_documents=True,
    #memory = memory, 
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [40]:
result = qa_chain({"query": "existe paridad hombre mujer en la propuesta de constitución?"})
print(result["result"])

/Users/ensamblador/Documents/nueva-constitucion-2023/.venv/lib/python3.11/site-packages/langchain/llms/bedrock.py:55: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human: 
You are a constitutional law assitant who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december 17th 2023.
(when asked for the source, provide the following link https://www.procesoconstitucional.cl/wp-content/uploads/2023/10/Texto-aprobado-Consejo-Constitucional_06.10.23.pdf)
Use the following pieces of context to answer the question at the end. Provide page number, article and letter if it is part of the answer. 
Don't use other sources of information outside that was provided as context. Don't user your prior knowledge about laws. Don't make up the answer using your intrinsic knowledge.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer. 
If the user doesn't ask a spe

 Sí, la propuesta de nueva Constitución establece la paridad de género en su artículo 6, numeral 3:

"El Estado promoverá la integración paritaria en los órganos del Estado y espacios de decisión política, y adoptará las medidas necesarias para asegurar la participación y representación política de las personas de todos los géneros en dichas instancias".

Este principio de paridad se aplica a varios organismos y cargos públicos según se indica en los siguientes artículos:

- Artículo 35, numeral 2: El Congreso de Diputadas y Diputados se compone de un número no inferior a 155 miembros electos en votación directa por distritos electorales. La ley determinará el número de integrantes, los distritos electorales y la forma de su elección, atendiendo el criterio de proporcionalidad. El sistema electoral deberá garantizar que los órganos deliberativos tengan una composición paritaria.

- Artículo 136, numeral 3: El Consejo de la Justicia se compone de diecisiete integrantes designados confor

In [171]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    ai_prefix= "A",
    human_prefix="H"
)

retriever=vectordb.as_retriever(search_type = "mmr", similarity_top_k=5, )
qa = ConversationalRetrievalChain.from_llm(
    llm,
    verbose=True,
    retriever=retriever,
    # condense_question_prompt=QA_CHAIN_PROMPT,
    #return_source_documents=True,
    #combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT},
    memory=memory,
)

In [172]:
template = """
You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)
qa.combine_docs_chain.llm_chain.prompt = QA_CHAIN_PROMPT

In [173]:
print(qa.question_generator.prompt.template)

Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:


In [174]:
print(qa.combine_docs_chain.llm_chain.prompt.template)


You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

{context}
Question: {question}
Helpful Answer:


In [178]:
qa.run({"question": "se protege el derecho a la educación?"})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: se protege el derecho a la educación?
Assistant:  Sí, la nueva constitución protege el derecho a la educación. En particular:

- Artículo 24, letra a) establece la libertad de enseñanza, que comprende el derecho a abrir establecimientos educacionales y desarrollar proyectos e idearios educativos.

- Artículo 24, letra b) señala que la libertad de enseñanza existe para garantizar a las familias el derecho a educar a sus hijos y elegir el tipo de educación.  

- Artículo 24, letra i) establece que el Estado deberá sostener y coordinar una red pluralista de establecimientos educacionales de calidad en todos los niveles.

- Artículo 24, letra l) señala que es deber del Estado y comunidad educativa promover el desarrollo profesional de docentes.

Espero haber podido

/Users/ensamblador/Documents/nueva-constitucion-2023/.venv/lib/python3.11/site-packages/langchain/llms/bedrock.py:50: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human: Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: se protege el derecho a la educación?

Assistant:  Sí, la nueva constitución protege el derecho a la educación. En particular:

- Artículo 24, letra a) establece la libertad de enseñanza, que comprende el derecho a abrir establecimientos educacionales y desarrollar proyectos e idearios educativos.

- Artículo 24, letra b) señala que la libertad de enseñanza existe para garantizar a las familias el derecho a educar a sus hijos y elegir el tipo de educación.  

- Artículo 24, letra i) establece que el Estado deberá sostener y coordinar una red pluralista de establecimientos educacionales de calidad en todos l


> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

16 
  
Solo en virtud de una ley de quorum  calificado, y siempre que no signifique tal 
discriminación, se podrán autorizar determinados beneficios directos o indirectos en favor de 
algún sector, actividad o zona geográfica, o establecer gravámenes especiales que afecten a 
uno u otras. En el caso de las franquicias o beneficios  indirectos, la estimación del costo de 
estos deberá incluirse anualmente en la Ley

' Sí, la nueva constitución protege el derecho al emprendimiento.\n\nEn el artículo 16, letra 34, se establece la "libertad para adquirir el dominio de toda clase de bienes, excepto aquellos que la naturaleza ha hecho comunes a todas las personas o que deban pertenecer a la Nación toda y la ley lo declare así". Esto incluye la libertad para emprender negocios y adquirir bienes necesarios para ello. \n\nAsimismo, en el mismo artículo y letra, se indica que "una ley de quorum calificado, cuando así lo exija el interés nacional, podrá establecer limitaciones o requisitos para la adquisición del dominio de algunos bienes". Esto significa que el emprendimiento está protegido, a menos que exista una ley especial que establezca restricciones por el interés nacional.\n\nEspero haber respondido de manera útil su consulta sobre la protección del derecho al emprendimiento en la nueva constitución. Por favor, indíqueme si requiere alguna aclaración adicional.'

In [175]:
result = qa({"question": "se protege el derecho a la educación?"})
print (result['answer'])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

científica y tecnológica, la creación artística y la protección e incremento del 
patrimonio cultural de la Nación.  
 
l) Es deber del Estado y de toda comunidad educativa promover el desarrollo 
profesional y respeto de los docentes y asistentes de la educación.  
 
24. La libertad de enseñanza.  
 
a) La libertad de enseñanza comprende el derecho de las personas de abrir, organizar y 
mantener establecimientos educacionales, así 

In [176]:
result = qa({"question": "y al emprendimiento?"})
print (result['answer'])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: se protege el derecho a la educación?
Assistant:  Sí, la nueva constitución protege el derecho a la educación. En particular:

- Artículo 24, letra a) establece la libertad de enseñanza, que comprende el derecho a abrir establecimientos educacionales y desarrollar proyectos e idearios educativos.

- Artículo 24, letra b) señala que la libertad de enseñanza existe para garantizar a las familias el derecho a educar a sus hijos y elegir el tipo de educación.  

- Artículo 24, letra i) establece que el Estado deberá sostener y coordinar una red pluralista de establecimientos educacionales de calidad en todos los niveles.

- Artículo 24, letra l) señala que es deber del Estado y comunidad educativa promover el desarrollo profesional de docentes.

Espero haber podido

/Users/ensamblador/Documents/nueva-constitucion-2023/.venv/lib/python3.11/site-packages/langchain/llms/bedrock.py:50: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human: Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: se protege el derecho a la educación?

Assistant:  Sí, la nueva constitución protege el derecho a la educación. En particular:

- Artículo 24, letra a) establece la libertad de enseñanza, que comprende el derecho a abrir establecimientos educacionales y desarrollar proyectos e idearios educativos.

- Artículo 24, letra b) señala que la libertad de enseñanza existe para garantizar a las familias el derecho a educar a sus hijos y elegir el tipo de educación.  

- Artículo 24, letra i) establece que el Estado deberá sostener y coordinar una red pluralista de establecimientos educacionales de calidad en todos l


> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

16 
  
Solo en virtud de una ley de quorum  calificado, y siempre que no signifique tal 
discriminación, se podrán autorizar determinados beneficios directos o indirectos en favor de 
algún sector, actividad o zona geográfica, o establecer gravámenes especiales que afecten a 
uno u otras. En el caso de las franquicias o beneficios  indirectos, la estimación del costo de 
estos deberá incluirse anualmente en la Ley

In [179]:
from langchain.chains.question_answering import load_qa_chain

In [ ]:
chain = load_qa_chain(BedrockChat, chain_type="stuff", memory=memory, verbose=True,)